In [1]:
!apt install tesseract-ocr libtesseract-dev
!pip install -q -U google-generativeai chromadb pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libarchive-dev libleptonica-dev tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  libarchive-dev libleptonica-dev libtesseract-dev tesseract-ocr tesseract-ocr-eng
  tesseract-ocr-osd
0 upgraded, 6 newly installed, 0 to remove and 45 not upgraded.
Need to get 8,560 kB of archives.
After this operation, 31.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libarchive-dev amd64 3.6.0-1ubuntu1.1 [582 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libleptonica-dev amd64 1.82.0-3build1 [1,562 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libtesseract-dev amd64 4.1.1-2.1build1 [1,600 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:5 http://arc

In [2]:
import time
from tqdm import tqdm
import pathlib
import google.generativeai as genai
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
import pandas as pd
from PIL import Image
import pytesseract
from IPython.display import Markdown

In [4]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

# Gemini Final Exercise

You're an astronomy student who's very curious about the Apollo 11 missions,
and through your research, you've found a lot of different types of data (otherwise known as multimodal) from NASA's
public archive.

1. Text: You have the full final NASA report post-mission, spanning over 300
pages of incredibly informative content that details a summary of everything
that happened as well as conclusions that NASA researchers and engineers
came to. For the sake of this exercise, we've selected 3 particularly interesting pages, and converted them to images (you'll see soon why).

2. Video: You also have several clips of the famous Neil Armstrong and Buzz Aldrin footage as they
first stepped onto the moon, containing highlights of their moonwalks as well
as raising the American flag.

3. Audio: Finally, you have highlights from the audio recorded throughout the
mission, which provides insights into how communication between the astronauts
occurred as well as from the astronauts to mission control.

Now, you want to search through and summarize this information for your
upcoming research paper. Using your newfound skills from this course, you
can accomplish this using Gemini! In particular, we will build a Retrieval Augmented Generation (RAG) system that you can directly interact with.

## Data Preparation

Before we begin, ensure that you've uploaded the resources.zip folder and unzipped it using the following command:

In [5]:
!wget -O resources.zip "https://video.udacity-data.com/topher/2024/June/66744e79_resources/resources.zip"

--2024-07-17 17:30:52--  https://video.udacity-data.com/topher/2024/June/66744e79_resources/resources.zip
Resolving video.udacity-data.com (video.udacity-data.com)... 104.19.141.72, 104.19.140.72, 104.19.139.72, ...
Connecting to video.udacity-data.com (video.udacity-data.com)|104.19.141.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 286142532 (273M) [application/zip]
Saving to: ‘resources.zip’

resources.zip       100%[===================>] 272.89M  8.05MB/s    in 5.1s    

2024-07-17 17:30:57 (53.3 MB/s) - ‘resources.zip’ saved [286142532/286142532]



In [6]:
!unzip resources.zip

Archive:  resources.zip
   creating: resources/
  inflating: __MACOSX/._resources    
   creating: resources/video/
  inflating: __MACOSX/resources/._video  
  inflating: resources/.DS_Store     
  inflating: __MACOSX/resources/._.DS_Store  
   creating: resources/audio/
  inflating: __MACOSX/resources/._audio  
   creating: resources/text/
  inflating: __MACOSX/resources/._text  
  inflating: resources/video/Apollo11PlaqueComparison.mov  
  inflating: __MACOSX/resources/video/._Apollo11PlaqueComparison.mov  
  inflating: resources/video/Apollo11Intro.mov  
  inflating: __MACOSX/resources/video/._Apollo11Intro.mov  
  inflating: resources/video/Apollo11MoonwalkMontage.mov  
  inflating: __MACOSX/resources/video/._Apollo11MoonwalkMontage.mov  
  inflating: resources/video/OneSmallStepCompilation.mov  
  inflating: __MACOSX/resources/video/._OneSmallStepCompilation.mov  
  inflating: resources/video/RaisingTheAmericanFlag.mov  
  inflating: __MACOSX/resources/video/._RaisingTheAmericanFl


As we saw throughout this course, when working with different types of data,
we first need to parse it in a way that Gemini can understand. We will prepare our data by extracting all file names from the `resources` directory.

In [7]:
data_dir = pathlib.Path("resources/")
all_file_names = [str(file) for file in data_dir.rglob("*") if file.is_file() and not file.name.startswith('.')]

In [8]:
for file_name in all_file_names:
    print(file_name)

print(len(all_file_names))

resources/video/BuzzDescendsCompilation.mov
resources/video/Apollo11Intro.mov
resources/video/Apollo11MoonwalkMontage.mov
resources/video/Apollo11PlaqueComparison.mov
resources/video/OneSmallStepCompilation.mov
resources/video/RaisingTheAmericanFlag.mov
resources/text/images-020.jpg
resources/text/images-333.jpg
resources/text/images-023.jpg
resources/audio/Apollo11OnboardAudioHighlightClip3.mp3
resources/audio/Apollo11OnboardAudioHighlightClip2.mp3
resources/audio/Apollo11OnboardAudioHighlightClip5.mp3
resources/audio/Apollo11OnboardAudioHighlightClip4.mp3
resources/audio/Apollo11OnboardAudioHighlightClip1.mp3
14


You should expect to see 14 files.

## Retrieval Augmented Generation (RAG)

To showcase how we build a RAG, we will first build one for the Text case, and generalize it further after. Here is the general idea:
1. **Data Preparation** (done above): We first collected various types of data from NASA's public archive related to the Apollo 11 mission, including text, video, and audio files.
2. **Data Extraction and Summarization**: Extract the multimodal data from images, e.g. extract text from images using Optical Character Recognition (OCR), and use Gemini to generate summaries using a specialized prompt.
3. **Embedding Generation**: Convert the generated summaries into vector embeddings using Gemini's Text Embedding Model. These embeddings represent the summaries in a numerical format suitable for efficient similarity searches.
4. **Creating a Vector Database**: A Vector database was created to store the embeddings. This database facilitates fast and efficient retrieval of relevant documents based on similarity searches. We chose to use Chroma DB.
5. **Querying the RAG System**: For a given query, the system retrieves the most relevant documents (based on their embeddings) and generates a response using the retrieved documents as context.

Something important to note is that RAGs are usually used only when there is a surplus of data. In other words, if the data can't fit into the model prompt. In this case, the data we provided likely can fit into Gemini's 1 million token window, but for the sake of simplicity and restrictions of Google Colab's runtime, we opted to use a smaller set of data.

### Text

We will use Tesseract OCR (Optical Character Recognition) to extract text from images of the NASA report.

In [9]:
pytesseract.pytesseract.tesseract_cmd = (r'/usr/bin/tesseract')

Let's create a function to take in our images of a PDF, transcribe them into text, and summarize each of them.

In [10]:
def create_text_summary():
  path = pathlib.Path("resources/text")

  text_summary_prompt = f"""You are an assistant tailored for summarizing text for retrieval.
  These summaries will be turned into vector embeddings and used to retrieve the raw text.
  Give a concise summary of the text that is well optimized for retrieval. Here is the text."""

  images = []
  text_summaries = []

  images = []
  text_summaries = []

  for f in path.glob("*"):
    if f.is_dir() or f.name.startswith('.'):
      continue

    image = Image.open(f)
    response = model.generate_content([text_summary_prompt, pytesseract.image_to_string(image)])

    images.append(image)
    text_summaries.append(response.text)

  return images, text_summaries

In [11]:
safety_settings = [
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",
    },
]

model = genai.GenerativeModel('models/gemini-1.5-flash', safety_settings=safety_settings)

In [12]:
image_files, text_summaries = create_text_summary()

Now, we can check out the generated summaries of the three pages we have!

In [13]:
for text_summary in text_summaries:
  print(text_summary)

This document is the Apollo 11 Flight Plan, outlining the mission's operations and crew activities to achieve the G Type Mission Lunar Landing objectives.  It uses trajectory parameters for a July 16, 1969 launch.  Changes to the Flight Plan must be submitted to the Crew Procedures Control Board (CPCB) for review and approval.  Contact Mr. W. J. North for distribution list updates. 

This document outlines ground rules and assumptions for an EPS analysis. It covers battery usage, S-band equipment operation, rendezvous radar, primary navigation and guidance subsystem, and forward window heaters. 

This document outlines the Apollo 11 mission launch and lunar orbit insertion (LOI) phases. Key events include: launch at 9:32 EDT on July 16, 1969, Earth orbit insertion at 11 min. 43 sec., translunar injection (TLI) at 2:44:26 GET, transposition, docking and LM ejection, separation from SIVB and an evasive maneuver, SIVB propulsive venting, cislunar navigation sightings, and four midcourse c

We create the Chroma database using the generated summaries. You might be wondering what Vector DB and Chroma DB are.

**Vector Database**: A specialized database designed to store and manage high-dimensional vectors, which are numerical representations of data points. It allows efficient similarity searches to find vectors (and their corresponding data) that are close to a given query vector.

**Chroma DB**: An implementation of a vector database used to store and retrieve vector embeddings. These embeddings are generated from our summaries and allow us to perform efficient similarity searches.


In [14]:
class GeminiEmbeddingFunction(EmbeddingFunction):
  def __call__(self, input: Documents) -> Embeddings:
    model = 'models/text-embedding-004'
    title = "Custom query"
    return genai.embed_content(model=model,
                                content=input,
                                task_type="retrieval_document",
                                title=title)["embedding"]

In [15]:
def create_chroma_db(documents, name):
  chroma_client = chromadb.Client()
  db = chroma_client.get_or_create_collection(name=name, embedding_function=GeminiEmbeddingFunction())


  for i, d in enumerate(documents):
    db.add(
      documents=d,
      ids=str(i)
    )
  return db

In [16]:
text_db = create_chroma_db(text_summaries, "text_nasa")

Let's also take a peak at the `text_db` and ensure that embeddings were generated:

In [17]:
data = {
    'embeddings': text_db.peek()['embeddings'],
    'documents': text_db.peek()['documents']
}

df = pd.DataFrame.from_dict(data, orient='index').transpose()
df

,embeddings,documents
0,"[0.06936851888895035, 0.010635172948241234, 0....","This document is the Apollo 11 Flight Plan, ou..."
1,"[0.08539081364870071, 0.008570509031414986, -0...",This document outlines ground rules and assump...
2,"[0.07238481193780899, -0.026040291413664818, 0...",This document outlines the Apollo 11 mission l...


You should see a column called `embeddings` with what are seemingly random values, but these values are actually high-dimensional vectors that represent the semantic meaning of your summaries.

Now let's actually try querying our information. We'll test a simple example like getting some file that has to do with the Apollo 11 Flight Plan.

In [18]:
def get_relevant_files(query, db):
  results = db.query(query_texts=[query], n_results=3)
  return results["ids"][0]

In [19]:
files = get_relevant_files("Apollo 11 Flight Plan", text_db)
print(files)

['0', '2', '1']


You should expect to see something like `['1', '0', '2']`. This means that the first entry in the `text_db` is most similar. If you look above at our `pd.DataFrame` output, the document with id 1 is the document about the Apollo 11 Flight Plan, so this is working as we expected!

### Video and Audio

Congrats! You've successfully built a working RAG for text. Now, let's extend this concept to Video and Audio, and build out some more complex queries. We'll begin by generalizing the above summary creation function to all sorts of modalities.

In [20]:
def create_summary(modality):
  path = data_dir / modality

  summary_prompt = f"""You are an assistant tailored for summarizing {modality} for retrieval.
  These summaries will be turned into vector embeddings and used to retrieve the raw {modality}.
  Give a concise summary of the {modality} that is well optimized for retrieval. Here is the {modality}."""

  files = []
  summaries = []

  for f in path.glob("*"):
    if f.is_dir() or f.name.startswith('.'):
      continue
    print(f)

    if modality == "text":
      file = Image.open(f)
      response = model.generate_content([summary_prompt, pytesseract.image_to_string(file)])

    else:
      file = genai.upload_file(f)

      while file.state.name == "PROCESSING":
        print("Waiting for video file upload...\n", end='')
        time.sleep(5)
        file = genai.get_file(file.name)

      response = model.generate_content([summary_prompt, file])

    files.append(file)
    summaries.append(response.text)

  return files, summaries

Now, we will create a folder with all of our data of different modalities. In particular, the first 5 are audio files, next 3 are text files, and final 6 are video files.

In [21]:
all_files = []
all_summaries = []

for modality_type in ["audio", "text", "video"]:
  files, summaries = create_summary(modality_type)
  all_files.extend(files)
  all_summaries.extend(summaries)

resources/audio/Apollo11OnboardAudioHighlightClip3.mp3
resources/audio/Apollo11OnboardAudioHighlightClip2.mp3
resources/audio/Apollo11OnboardAudioHighlightClip5.mp3
resources/audio/Apollo11OnboardAudioHighlightClip4.mp3
resources/audio/Apollo11OnboardAudioHighlightClip1.mp3
resources/text/images-020.jpg
resources/text/images-333.jpg
resources/text/images-023.jpg
resources/video/BuzzDescendsCompilation.mov
Waiting for video file upload...
resources/video/Apollo11Intro.mov
Waiting for video file upload...
resources/video/Apollo11MoonwalkMontage.mov
Waiting for video file upload...
resources/video/Apollo11PlaqueComparison.mov
Waiting for video file upload...
resources/video/OneSmallStepCompilation.mov
Waiting for video file upload...
resources/video/RaisingTheAmericanFlag.mov
Waiting for video file upload...


In [22]:
db = create_chroma_db(all_summaries, "nasa")

Again, ensure that the embeddings were generated. Notice that now, we have audio, video, and text data.

In [23]:
data = {
    'embeddings': db.peek()['embeddings'],
    'documents': db.peek()['documents']
}

df = pd.DataFrame.from_dict(data, orient='index').transpose()
df

,embeddings,documents
0,"[0.043420735746622086, -0.032168611884117126, ...",A group of astronauts are discussing their mis...
1,"[0.07031884044408798, -0.01165339071303606, -0...",The audio is a conversation between two people...
2,"[0.01251338329166174, -0.018983716145157814, -...",Buzz Aldrin describes the lunar surface from t...
3,"[0.033819299191236496, -0.005007217638194561, ...",Apollo 11 astronauts Buzz Aldrin and Neil Arms...
4,"[0.023345697671175003, -0.015683135017752647, ...",Neil Armstrong takes his first step on the moo...
5,"[0.04685447737574577, -0.013304195366799831, -...",A comparison of two versions of the Apollo 11 ...
6,"[0.02972055785357952, -0.0047592585906386375, ...",Two people are having a conversation about fli...
7,"[0.022077776491642, -0.0004612583725247532, 0....","Aircraft pilots discussing flight plan, weathe..."
8,"[0.011432270519435406, 0.009073207154870033, 0...",Two people are discussing flying a plane. They...
9,"[0.07731004804372787, -0.0017914429772645235, ...",This Flight Plan outlines the Apollo 11 missio...


In [24]:
files = get_relevant_files("communication with Mission Control", db)
print(files)

['1', '5', '4']


Can we do more than just return the most relevant file? Yes we can! We can ask Gemini to return a response to the query using the files it thinks are most relevant, provide an answer and tell us what files it used! This is really exciting, and has vast applications in many industries.

In [25]:
def query_rag(query, db):
    files = get_relevant_files(query, db)
    prompt = [all_files[int(f)] for f in files]
    prompt.append("Generate a response to the query using the provided files. Here is the query.")
    prompt.append(query)
    return model.generate_content(prompt).text, [all_file_names[int(f)] for f in files]

In [26]:
for response in query_rag("Explain what happened with the Apollo 11 Mission.", db):
    print(response)

Apollo 11 was the spaceflight that landed the first humans on the Moon.  Neil Armstrong and Buzz Aldrin landed their lunar module, the Eagle, on July 20, 1969, at 20:17:40 GMT.  Armstrong became the first person to walk on the lunar surface six hours later at 02:56 GMT on July 21, 1969, with Aldrin joining him about 20 minutes later.  They spent about two hours outside the module collecting lunar samples, planting the U.S. flag, and taking photographs before returning to the lunar module and preparing for their ascent to rendezvous with the command module, Columbia, piloted by Michael Collins.  Armstrong and Aldrin lifted off from the moon's surface at 17:54 GMT on July 21, 1969.  The mission concluded on July 24, 1969, when Columbia splashed down in the Pacific Ocean. 

['resources/audio/Apollo11OnboardAudioHighlightClip3.mp3', 'resources/text/images-333.jpg', 'resources/audio/Apollo11OnboardAudioHighlightClip1.mp3']


In [27]:
for response in query_rag("What happens at the Translunar Coast in the Mission Description?", db):
    print(response)

After TLI, which places the spacecraft in a free lunar return trajectory, the following major events occur prior to LOI: transposition, docking and LM ejection, including SIVB photography. 
Separation from SIVB and a CSM evasive maneuver. 
SIVB propulsive venting of propellants (slingshot). 
Two series of P23 cislunar navigation sightings, star/earth horizon, consisting of five sets at 06:00 GET and five sets at 24:30 GET.
Four midcourse corrections which take place at TLI + 9, TLI + 24, LOI - 22 and LOI - 5 hours with ΔV nominally zero. 
['resources/text/images-333.jpg', 'resources/video/RaisingTheAmericanFlag.mov', 'resources/video/BuzzDescendsCompilation.mov']


In [30]:
for response in query_rag("What happens at the Translunar Coast in the Mission Description?", db):
    print(response)

The Translunar Coast takes place from 73:10 to 75:54 GET.  After TLI (the Translunar Injection burn), the spacecraft is on a free lunar return trajectory. The mission description details the major events that occur prior to LOI, including SIVB photography, separation of the SIVB, and a CSM evasive maneuver, SIVB propulsive venting of propellants, P23 cislunar navigation sightings, and four midcourse corrections. 
['resources/text/images-333.jpg', 'resources/video/RaisingTheAmericanFlag.mov', 'resources/video/BuzzDescendsCompilation.mov']
